In [ ]:
!pip install  blobfile tqdm pyYaml pillow    # e.g. torch 1.7.1+cu110.

In [2]:
!pip install --upgrade gdown && bash ./download.sh

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
--2025-12-27 00:43:06--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_classifier.pt
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.241.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.241.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216496432 (206M) [application/octet-stream]
Saving to: ‘256x256_classifier.pt.1’

256x256_classifier. 100%[===================>] 206.47M  4.20MB/s    in 50s     

2025-12-27 00:43:58 (4.10 MB/s) - ‘256x256_classifier.pt.1’ saved [216496432/216496432]

--2025-12-27 00:43:58--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion.pt
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.241.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.241.33|:443.

In [1]:
!python test.py --conf_path confs/face_example.yml

Start face_example
sampling...
100%|███████████████████████████████████████| 4570/4570 [06:43<00:00, 11.32it/s]
sampling complete


In [2]:
!pwd

/root/autodl-tmp/Repaint/RePaint


In [3]:
!mkdir -p "my_input/images"

In [3]:
!curl -L -o my_input/images/natural-images.zip https://www.kaggle.com/api/v1/datasets/download/prasunroy/natural-images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  342M  100  342M    0     0  2169k      0  0:02:41  0:02:41 --:--:-- 12.2M


In [ ]:
!unzip dataset.zip -d  my_input/images

Archive:  my_input/images/natural-images.zip
  inflating: my_input/images/data/natural_images/airplane/airplane_0000.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0001.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0002.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0003.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0004.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0005.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0006.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0007.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0008.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0009.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0010.jpg  
  inflating: my_input/images/data/natural_images/airplane/airplane_0011.jpg  
  inflating: my_inp

In [ ]:
%%writefile generate_mask.py
import os
import argparse
from PIL import Image, ImageDraw
import random

def create_mask(image_path, output_dir, mask_type='box'):
    try:
        img = Image.open(image_path).convert('RGB')
        w, h = img.size

        # 白色背景 (255) = 保留原图区域
        # 黑色 (0) = 需要修复(Inpaint)的区域
        mask = Image.new('L', (w, h), 255)
        draw = ImageDraw.Draw(mask)

        if mask_type == 'box':
            # 在中心创建一个黑色矩形框 (大约占图像的50%)
            box_w, box_h = w // 2, h // 2
            x1 = (w - box_w) // 2
            y1 = (h - box_h) // 2
            draw.rectangle((x1, y1, x1 + box_w, y1 + box_h), fill=0)

        elif mask_type == 'random':
            # 随机线条遮罩
            for _ in range(5):
                x1 = random.randint(0, w)
                y1 = random.randint(0, h)
                x2 = random.randint(0, w)
                y2 = random.randint(0, h)
                width = random.randint(w // 20, w // 10)
                draw.line((x1, y1, x2, y2), fill=0, width=width)

        filename = os.path.basename(image_path)
        save_path = os.path.join(output_dir, filename)
        mask.save(save_path)
        print(f"Generated mask for {filename} -> {save_path}")

    except Exception as e:
        print(f"Skipping {image_path}: {e}")

def main():
    parser = argparse.ArgumentParser(description="生成Inpainting所需的Mask掩码图像")
    parser.add_argument('--image_dir', type=str, required=True, help='包含源图像的文件夹路径')
    parser.add_argument('--output_dir', type=str, required=True, help='保存生成的Mask的文件夹路径')
    parser.add_argument('--type', type=str, default='box', choices=['box', 'random'], help='Mask类型: "box"(矩形) 或 "random"(随机线条)')
    args = parser.parse_args()

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    # 支持的图片扩展名
    valid_exts = ['.jpg', '.jpeg', '.png', '.bmp', '.webp']

    # 筛选出符合扩展名的文件
    files = [f for f in os.listdir(args.image_dir) if os.path.splitext(f)[1].lower() in valid_exts]

    if not files:
        print(f"No images found in {args.image_dir}")
        return

    print(f"Found {len(files)} images. Generating masks...")
    # 遍历所有图片并生成对应的Mask
    for f in files:
        create_mask(os.path.join(args.image_dir, f), args.output_dir, args.type)

    print("Done.")

if __name__ == '__main__':
    main()


Writing generate_mask.py


**写自定义配置文件**
主要参数:  
1. `model_path`: 模型路径  
2. `data`: 数据集配置  
   `gt_path`: 自定义原图数据集路径  
   `mask_path`: 自定义遮罩数据集路径  
   （以上两者是一一对应的）  
   `srs`: 修复图片的输出文件夹  
   `lrs`: 原图被遮罩后的图片输出文件夹  
   `gts`: 原图输出文件夹  
   `gt_keep_masks`: 保存遮罩的文件夹  
   `batch_size`: 批处理大小，一次处理几张图片  

In [ ]:
%%writefile confs/user_custom.yml

attention_resolutions: 32,16,8
class_cond: false
diffusion_steps: 1000
learn_sigma: true
noise_schedule: linear
num_channels: 256
num_head_channels: 64
num_heads: 4
num_res_blocks: 2
resblock_updown: true
use_fp16: false
use_scale_shift_norm: true
classifier_scale: 4.0
lr_kernel_n_std: 2
num_samples: 100
show_progress: true
timestep_respacing: '250'
use_kl: false
predict_xstart: false
rescale_timesteps: false
rescale_learned_sigmas: false
classifier_use_fp16: false
classifier_width: 128
classifier_depth: 2
classifier_attention_resolutions: 32,16,8
classifier_use_scale_shift_norm: true
classifier_resblock_updown: true
classifier_pool: attention
num_heads_upsample: -1
channel_mult: ''
dropout: 0.0
use_checkpoint: false
use_new_attention_order: false
clip_denoised: true
use_ddim: false
latex_name: RePaint
method_name: Repaint
image_size: 256
model_path: ./data/pretrained/celeba256_250000.pt
name: my_custom_viz
inpa_inj_sched_prev: true
n_jobs: 1
print_estimated_vars: true
inpa_inj_sched_prev_cumnoise: false
schedule_jump_params:
  t_T: 250
  n_sample: 1
  jump_length: 10
  jump_n_sample: 10
data:
  eval:
    custom_data:
      mask_loader: true
      gt_path: ./my_input/images
      mask_path: ./my_input/masks
      image_size: 256
      class_cond: false
      deterministic: true
      random_crop: false
      random_flip: false
      return_dict: true
      drop_last: false
      batch_size: 4
      return_dataloader: true
      offset: 0
      paths:
        srs: ./log/my_custom_viz/inpainted
        lrs: ./log/my_custom_viz/gt_masked
        gts: ./log/my_custom_viz/gt
        gt_keep_masks: ./log/my_custom_viz/gt_keep_mask


Writing confs/user_custom.yml


In [ ]:
%%writefile visualize.py

# Copyright (c) 2022 Huawei Technologies Co., Ltd.
# Licensed under CC BY-NC-SA 4.0 (Attribution-NonCommercial-ShareAlike 4.0 International) (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode
#
# The code is released for academic research use only. For commercial use, please contact Huawei Technologies Co., Ltd.
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import argparse
import torch as th
import torch.nn.functional as F
import time
import conf_mgt
from utils import yamlread
from guided_diffusion import dist_util
from utils import imwrite # 显式导入用于保存图像的函数

# Workaround
try:
    import ctypes
    libgcc_s = ctypes.CDLL('libgcc_s.so.1')
except:
    pass


from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    classifier_defaults,
    create_model_and_diffusion,
    create_classifier,
    select_args,
)  # noqa: E402

def toU8(sample):
    if sample is None:
        return sample

    sample = ((sample + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample = sample.permute(0, 2, 3, 1)
    sample = sample.contiguous()
    sample = sample.detach().cpu().numpy()
    return sample


def main(conf: conf_mgt.Default_Conf, save_every=10):
    """
    主函数: 执行图像生成可视化
    :param conf:配置对象，包含模型和运行参数
    :param save_every: 每隔多少步保存中间结果
    """

    print("Start Visualization", conf['name'])

    device = dist_util.dev(conf.get('device'))

    # 创建模型和扩散过程
    # create_model_and_diffusion: 根据配置初始化UNet模型和高斯扩散过程
    model, diffusion = create_model_and_diffusion(
        **select_args(conf, model_and_diffusion_defaults().keys()), conf=conf
    )
    model.load_state_dict(
        dist_util.load_state_dict(os.path.expanduser(
            conf.model_path), map_location="cpu")
    )
    model.to(device)
    if conf.use_fp16:
        model.convert_to_fp16()
    model.eval()

    show_progress = conf.show_progress

    if conf.classifier_scale > 0 and conf.classifier_path:
        print("loading classifier...")
        # 如果启用了分类器引导(classifier guidance)，则加载分类器模型
        classifier = create_classifier(
            **select_args(conf, classifier_defaults().keys()))
        classifier.load_state_dict(
            dist_util.load_state_dict(os.path.expanduser(
                conf.classifier_path), map_location="cpu")
        )

        classifier.to(device)
        if conf.classifier_use_fp16:
            classifier.convert_to_fp16()
        classifier.eval()

        def cond_fn(x, t, y=None, gt=None, **kwargs):
            # 定义引导函数：计算分类器梯度以引导生成过程
            assert y is not None
            with th.enable_grad():
                x_in = x.detach().requires_grad_(True)
                logits = classifier(x_in, t)
                log_probs = F.log_softmax(logits, dim=-1)
                selected = log_probs[range(len(logits)), y.view(-1)]
                # 返回特定类别的梯度 * scale
                return th.autograd.grad(selected.sum(), x_in)[0] * conf.classifier_scale
    else:
        cond_fn = None

    def model_fn(x, t, y=None, gt=None, **kwargs):
        # 模型前向传播函数
        assert y is not None
        return model(x, t, y if conf.class_cond else None, gt=gt)

    print("sampling...")

    dset = 'eval'
    eval_name = conf.get_default_eval_name()
    # 获取数据加载器
    dl = conf.get_dataloader(dset=dset, dsName=eval_name)

    for batch in iter(dl):
        for k in batch.keys():
            if isinstance(batch[k], th.Tensor):
                batch[k] = batch[k].to(device)

        model_kwargs = {}
        model_kwargs["gt"] = batch['GT']

        gt_keep_mask = batch.get('gt_keep_mask')
        if gt_keep_mask is not None:
            model_kwargs['gt_keep_mask'] = gt_keep_mask

        batch_size = model_kwargs["gt"].shape[0]

        if conf.cond_y is not None:
            classes = th.ones(batch_size, dtype=th.long, device=device)
            model_kwargs["y"] = classes * conf.cond_y
        else:
            classes = th.randint(
                low=0, high=NUM_CLASSES, size=(batch_size,), device=device
            )
            model_kwargs["y"] = classes

        # 选择逐步采样函数 (Progressive Sampling)
        if conf.use_ddim:
            if hasattr(diffusion, 'ddim_sample_loop_progressive'):
                sample_fn_progressive = diffusion.ddim_sample_loop_progressive
            else:
                print("Warning: DDIM progressive sampling not found, falling back to P-Sample progressive.")
                sample_fn_progressive = diffusion.p_sample_loop_progressive
        else:
            sample_fn_progressive = diffusion.p_sample_loop_progressive

        # 逐步生成样本
        generator = sample_fn_progressive(
            model_fn,
            (batch_size, 3, conf.image_size, conf.image_size),
            clip_denoised=conf.clip_denoised,
            model_kwargs=model_kwargs,
            cond_fn=cond_fn,
            device=device,
            progress=show_progress,
            conf=conf
        )

        # 中间结果输出目录
        out_base_path = conf['data'][dset][eval_name]['paths']['srs']
        intermediate_path = os.path.join(out_base_path, "intermediate")
        os.makedirs(intermediate_path, exist_ok=True)

        img_name_base = batch['GT_name'][0].split('.')[0] # 假设batch size为1，简化命名

        print(f"Generating for {img_name_base}...")

        step = 0
        for result in generator:
            step += 1
            if step % save_every == 0 or step == 1:
                sample = result['sample'] # This is x_t
                # You can also use result['pred_xstart'] for the denoised prediction

                srs = toU8(sample)

                # 保存中间步骤的图像
                # 假设batch size为1
                img_out_name = f"{img_name_base}_step_{step:04d}.png"
                full_path = os.path.join(intermediate_path, img_out_name)

                # imwrite 期望 [H, W, C] 格式的 numpy uint8 数组，toU8 函数已经转换好了
                imwrite(img=srs[0], path=full_path)

        # 保存最终结果，与原始脚本一致
        srs = toU8(result['sample'])
        gts = toU8(result['gt'])
        # 计算低分辨率或被遮挡的图像 (LRS)
        lrs = toU8(result.get('gt') * model_kwargs.get('gt_keep_mask') + (-1) *
                   th.ones_like(result.get('gt')) * (1 - model_kwargs.get('gt_keep_mask')))
        gt_keep_masks = toU8((model_kwargs.get('gt_keep_mask') * 2 - 1))

        # 调用 conf.eval_imswrite 保存所有相关图像 (GT, Mask, Input, Output)
        conf.eval_imswrite(
            srs=srs, gts=gts, lrs=lrs, gt_keep_masks=gt_keep_masks,
            img_names=batch['GT_name'], dset=dset, name=eval_name, verify_same=False)

    print("Sampling and visualization complete.")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--conf_path', type=str, required=False, default=None, help='配置文件路径')
    parser.add_argument('--save_every', type=int, default=10, help="每N步保存一次中间图像")
    args = vars(parser.parse_args())

    conf_arg = conf_mgt.conf_base.Default_Conf()
    conf_arg.update(yamlread(args.get('conf_path')))
    main(conf_arg, save_every=args.get('save_every'))


Writing visualize.py


In [ ]:
!python generate_mask.py --image_dir ./my_input/images --output_dir ./my_input/masks --type random

Found 2 images. Generating masks...
Generated mask for wew.jpg -> ./my_input/masks/wew.jpg
Generated mask for bradpit.jpg -> ./my_input/masks/bradpit.jpg
Done.


**如果下一步运行过程中出现images和masks数量不匹配，运行该条指令**

In [ ]:
!rm -rf ./my_input/masks/.ipynb_checkpoints

In [ ]:
!python visualize.py --conf_path confs/user_custom.yml --save_every 20

Start Visualization my_custom_viz
sampling...
Generating for bradpit...
100% 4570/4570 [07:06<00:00, 10.73it/s]
Generating for wew...
100% 4570/4570 [07:07<00:00, 10.69it/s]
Sampling and visualization complete.
